In [ ]:
  # Fix the strike price 
# Main thing 
# Create samples such that ;
# '''
# The four variables make a sample x is a data point x = (t, S, r, sigma) is data points. 
# No stable results -> then normalise the varaibles. 
#        t -> (0,2) only discrete number of days / 365. T is measure in years. 
#        S -> (50,150) stock prices. 
#        r -> [0%,1%] 
#        sigma (volatility) -> [1%,50%] -> standard deviation of the log of the returns. 
# We will use uniform distribution. 
# '''



In [ ]:
pip install pytorch-lightning # https://pytorch-lightning.readthedocs.io/en/latest/common/optimizers.html - optimization to manually turn off pytorch. 


In [ ]:
import numpy as np
import scipy.stats as si
import sympy as sy
from sympy.stats import Normal, cdf
from sympy import init_printing
init_printing()
import torch 
from torch import autograd
from torch import nn # import neural network 
from torch import optim 
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader
import pytorch_lightning as pl

is_cuda = torch.cuda.is_available()

if is_cuda:
  device = torch.device("cuda")
  print("GPU is available to use")

else:
  device = torch.device("cpu")
  print( " GPU not availiable to use, CPU used")


GPU is available to use


In [ ]:
# Increase data up . # data works for 500 samples. 

In [ ]:
# stock price - between [50,150]

# stock price - between [50,150]
torch.set_printoptions(precision =8) # https://pytorch.org/docs/stable/generated/torch.set_printoptions.html - gives us values for options
a,b = 2000 ,1 #dimension of the pytorch tensor to be generated
low , high = 50 , 150 #range of uniform distribution
torch.manual_seed(50) #Fx the values
S_t = torch.distributions.uniform.Uniform(low, high).sample([a,b]) # samples from low and high values. a,b are the dimensions

Stock = S_t.view(2000,1)
print(Stock.shape)

torch.Size([2000, 1])


In [ ]:

# Time -[ between 0 and 2]
torch.set_printoptions(precision =8) # fixes the precision
a,b = 2000 ,1 #dimension of the pytorch tensor to be generated
low_t , high_t = 0, 730 #range of uniform distribution by number of days [0, 2]Years
torch.manual_seed(56) #Fx the values
t_t= torch.randint(low_t, high_t, size= (2000,1) )  # https://pytorch.org/docs/stable/generated/torch.randint.html - generates uniformaly integers

t = t_t / 365 # to get a more reliable spread of results

# time for maturity 
#print(t)
Time = t.view(2000,1)
print(Time.shape)

torch.Size([2000, 1])


In [ ]:
torch.set_printoptions(precision =8)
a,b = 2000 ,1 #dimension of the pytorch tensor to be generated
low , high = 0 , 0.01 #range of uniform distribution [0%,1%]
torch.manual_seed(12) #Fx the values
r = torch.distributions.uniform.Uniform(low, high).sample([a,b]) # samples from low and high values. a,b are the dimensions

Rate = r.view(2000,1)
print(Rate.shape)

torch.Size([2000, 1])


In [ ]:
# volatility 
torch.set_printoptions(precision =8)
a,b = 2000 ,1 #dimension of the pytorch tensor to be generated
low , high =  0.01, 0.5 #range of uniform distribution [1%,50%]
torch.manual_seed(30) #Fx the values
si = torch.distributions.uniform.Uniform(low, high).sample([a,b]) # samples from low and high values. a,b are the dimensions
# whenevr we want to fix a distribution we just use torch.manual_seed to fix the distribution.
Sigma = si.view(2000,1)
print(Sigma.shape) # 500 samples by 1 column

torch.Size([2000, 1])


In [ ]:
# Stock, Time, Rate ,Sigma -> 4 features (variables)
# Strike
# https://discuss.pytorch.org/t/tensor-stack-or-concatenate/34331/6 - column and concatinate 

X = torch.column_stack([Stock, Time, Rate, Sigma]) # rate = 0 change
print(X)
print(X.shape) # m x n matrix 


tensor([[1.11802200e+02, 7.09589064e-01, 4.65687457e-03, 4.51364458e-01],
        [5.68693161e+01, 1.86849320e+00, 2.32766801e-03, 3.75717849e-01],
        [8.89275208e+01, 1.33424652e+00, 4.52720979e-03, 2.41090730e-01],
        ...,
        [7.55905457e+01, 1.68493152e+00, 2.76269601e-03, 4.49617684e-01],
        [6.16124725e+01, 1.87397265e+00, 7.82432314e-03, 1.40119523e-01],
        [6.70583801e+01, 6.71232879e-01, 1.49058105e-04, 1.10146537e-01]])
torch.Size([2000, 4])


In [ ]:
X.requires_grad_(True) # graident for backprop activated with requires_grad true
X.retain_grad() # leaf tensor is a root of the tree 

print(X.shape) 

torch.Size([2000, 4])


In [ ]:
# non- divident paying asset. 
# P + S - C = K * e^-r(T-t)
# Set strike price = 100

Strike = 100 # fixed hyper paramters. 

# Our case this is a fixed paramter. 

In [ ]:
def euro_vanilla_call(x):
  d1 = (torch.log(x[0] / Strike) + (x[2] + 0.5 * x[3] ** 2) * x[1]) / (x[3] * torch.sqrt(x[1]))
  d2 = (torch.log(x[0] / Strike) + (x[2] - 0.5 * x[3] ** 2) * x[1]) / (x[3] * torch.sqrt(x[1])) # time x[1] with resepct to the time left not time. (- theta)
    
  call_t = (x[0] * torch.distributions.normal.Normal(0,1).cdf(d1) - Strike * torch.exp(-x[2] * x[1]) * torch.distributions.normal.Normal(0,1).cdf(d2)) 
  
  call = torch.nn.functional.relu(call_t) # to get rid of any negative prices
  return call
# https://pytorch.org/docs/stable/distributions.html - Normal( loc,scale)
#https://www.itl.nist.gov/div898/handbook/eda/section3/eda364.htm - loc and scale the same as mean and standard deviation.
# https://pytorch.org/docs/stable/generated/torch.log.html - returns natrual logarithem
# https://pytorch.org/docs/stable/generated/torch.sqrt.html - returns squart root


In [ ]:
y = torch.zeros(size = (2000 , 1)) # samples

for i in range(X.shape[0]):
  y[i] = euro_vanilla_call(X[i]) # runs through each row computation. 

print(y) # M samples dimension all in all. 

tensor([[2.27208824e+01],
        [2.64448738e+00],
        [6.00817680e+00],
        ...,
        [1.02451553e+01],
        [3.49690318e-02],
        [6.31218427e-06]], grad_fn=<CopySlices>)


In [ ]:
f_o = torch.zeros(size = (2000,4))

# First order DY/DX

for i in range(X.shape[0]):
  f_o[i] = torch.autograd.functional.jacobian( euro_vanilla_call, X[i])

print(f_o.shape) # m samples by n features 
  
print(f_o) # call option. Theta is always negative as depreciation with the value of the option. And measures the variation.

torch.Size([2000, 4])
tensor([[6.88715994e-01, 1.08390903e+01, 3.85158386e+01, 3.32854576e+01],
        [2.02218562e-01, 2.22321677e+00, 1.65465202e+01, 2.19076653e+01],
        [3.97257090e-01, 3.71162462e+00, 3.91186676e+01, 3.96126518e+01],
        ...,
        [4.28696007e-01, 5.20033455e+00, 3.73384399e+01, 3.85174026e+01],
        [9.32326913e-03, 8.32694024e-02, 1.01093388e+00, 2.11440635e+00],
        [5.87166869e-06, 1.21777222e-04, 2.60057859e-04, 1.48352201e-03]])


In [ ]:
s_o = torch.zeros(size = (2000,4,4))

#Second Order Risk D2Y/ DX2

for i in range(X.shape[0]):
  s_o[i] = torch.autograd.functional.hessian(euro_vanilla_call , X[i])

print(s_o.shape) # m samples by n features by n features .
  
print(s_o) # take logarithems if really small. Monotonic.

torch.Size([2000, 4, 4])
tensor([[[ 8.31418484e-03, -2.35627536e-02,  6.59594297e-01, -8.76964852e-02],
         [-2.35628486e-02, -7.17481613e+00,  4.47184486e+01,  2.45463295e+01],
         [ 6.59594417e-01,  4.47184601e+01,  2.49975967e+01, -3.05762863e+01],
         [-8.76967907e-02,  2.45463295e+01, -3.05763168e+01,  4.06529236e+00]],

        [[ 9.64909233e-03,  1.02882445e-01,  1.02531207e+00,  1.01059127e+00],
         [ 1.02882467e-01,  8.94269198e-02,  1.56337624e+01,  1.25302544e+01],
         [ 1.02531230e+00,  1.56337605e+01,  7.80325623e+01,  6.64510422e+01],
         [ 1.01059103e+00,  1.25302505e+01,  6.64510345e+01,  6.54969406e+01]],

        [[ 1.55720217e-02,  8.41536969e-02,  1.84763956e+00,  8.62057924e-01],
         [ 8.41535926e-02, -1.12068129e+00,  3.43516273e+01,  1.70959949e+01],
         [ 1.84763968e+00,  3.43516235e+01,  1.67030777e+02,  4.94311371e+01],
         [ 8.62058163e-01,  1.70960007e+01,  4.94312134e+01,  2.30631104e+01]],

        ...,

       

In [ ]:
# Data Collection

x = X.type(torch.FloatTensor).to(device)
y = y.type(torch.FloatTensor).to(device)
f_o = f_o.type(torch.FloatTensor).to(device)
s_o = s_o.type(torch.FloatTensor).to(device)

print(X.shape)
print(y.shape)
print(f_o.shape)
print(s_o.shape)

torch.Size([2000, 4])
torch.Size([2000, 1])
torch.Size([2000, 4])
torch.Size([2000, 4, 4])


In [ ]:
# Pre-processing for pytorch. 

full_data = torch.utils.data.TensorDataset(X , y, f_o, s_o)
train, val, test = random_split(full_data, [1500, 300, 200]) # train, validation, test data. 
train_loader = DataLoader(train, batch_size = 128, shuffle=False , num_workers =0) # hardcore
val_loader = DataLoader(val, batch_size = 128 , shuffle = False , num_workers =0)
test_loader = DataLoader(test, batch_size = 128 , shuffle= False, num_workers =0)

#X_train = train
#X_val, y_val, f_o_val, s_o_val = val

In [ ]:
# Paramters and hyperparamters 
input_size = 4 # x features
hidden_size_1 = 256
hidden_size_2 = 128
hidden_size_3 = 64
output_size = 1 # y output of training 

#loss hyper- parameters 
a_1= 1
a_2 = 0.5
a_3 = 0.2

In [ ]:
class SecondOrderNetwork(pl.LightningModule):
  
  def __init__(self):
    super(SecondOrderNetwork,self).__init__()
    self.layer_1 = nn.Linear(input_size, hidden_size_1) # linear layer
    self.layer_2 = nn.Linear(hidden_size_1, hidden_size_2)
    self.layer_3 = nn.Linear(hidden_size_2, hidden_size_3)
    self.layer_4 = nn.Linear(hidden_size_3, output_size)

#   self.Sigmoid = nn.Sigmoid() # absolute 
    self.Softplus = nn.Softplus() # Softplus/ sigmoid
    self.loss = nn.MSELoss() # Possibility try different loss. Difference in log -> MSE(log([y]) - log([predicted_y)]) -> solves small

    self.automatic_optimization = False # Turn off automatic optimization. Turns off backprop automatically. 
# Automatic optimization = False does manual optimization not autoamtic - https://pytorch-lightning.readthedocs.io/en/latest/common/optimizers.html
  def forward(self, x):
    # layer 1
    x = self.layer_1(x)
    x = self.Softplus(x)

    # layer 2
    x = self.layer_2(x)
    x = self.Softplus(x)

    # layer 3 
    x = self.layer_3(x) # linear layer on final layer
    x = self.Softplus(x)
    #layer 4
    x = self.layer_4(x)

    return x
# 4 - layers MLP (see if it performs better then 3)

  def training_step(self, train_batch, batch_idx):
    opt = self.optimizers() # access optimizers thats defined below in optimization function
    opt.zero_grad() # clear gradients from the preavious training set (so only keeps paramters that updated)

    x, y, f_o, s_o = train_batch
    predicted_y = self.forward(x).to(device)
    # loop over batch
    predicted_f_o = torch.zeros(size=(x.shape[0], 4)).to(device)
    for i in range(x.shape[0]): # x_shape is the batch size
      predicted_f_o[i] = torch.autograd.functional.jacobian(self.forward, x[i], create_graph=True, strict=True) # https://github.com/pytorch/pytorch/issues/46918 - Can show we need create_graph to be true as it backpropagets through the graph

    predicted_s_o = torch.zeros(size=(x.shape[0], 4, 4)).to(device) # when creating new variable put it on device
    for i in range(x.shape[0]): # x_shape is the batch size
      predicted_s_o[i] = torch.autograd.functional.hessian(self.forward, x[i], create_graph=True, strict=True)
    # create_graph = True makes derivatives differentiable through the chain using back prop. 
    # strict = True makes sure all inputs are connected to the outputs ortherwise an error will occur. If false it will be all zeros.
    loss = a_1 * (1/y.shape[0])*self.loss(y, predicted_y) + a_2 * (1/f_o.shape[0])*self.loss( f_o, predicted_f_o) + a_3 *(1/s_o.shape[0])*self.loss(s_o,  predicted_s_o) # change loss hyperparameters

    loss.backward(retain_graph=True) # Saves gradeints and not deleted them after first backprop . 

    opt.step() # Every batch update your paramters using optimizer step. 
    
    # logs 
    tensorboard_logs = {'train_loss' : loss}
    self.log('train_loss', loss)
    #print(loss)
    return {'loss': loss, 'log' : tensorboard_logs, 'train_loss': loss}

  def training_epoch_end(self, outputs):
    avg_loss = torch.stack([x['train_loss'] for x in outputs]).mean()

    tensorboard_logs = {'train_loss': avg_loss}
    self.logger.experiment.add_scalar('train_loss',avg_loss, self.current_epoch) # https://stackoverflow.com/questions/66290662/how-to-get-step-wise-validation-loss-curve-over-all-epochs-in-pytorch-lightning 

    print('train_loss: ', avg_loss , "epoch_number",self.current_epoch)
    {'avg_train_loss': avg_loss,'progress_bar': tensorboard_logs} # do not return for training epoch_end
    
  def validation_step(self, val_batch, batch_idx):
    x, y, f_o, s_o = val_batch
    predicted_y = self.forward(x).to(device)
    # loop over batch
    predicted_f_o = torch.zeros(size=(x.shape[0], 4)).to(device)
    for i in range(x.shape[0]): # validatio batch size
      predicted_f_o[i] = torch.autograd.functional.jacobian(self.forward, x[i], create_graph=True, strict=True)

    predicted_s_o = torch.zeros(size=(x.shape[0], 4, 4)).to(device)
    for i in range(x.shape[0]): # validation batch size
      predicted_s_o[i] = torch.autograd.functional.hessian(self.forward, x[i], create_graph=True, strict=True)

    loss = a_1 * (1/y.shape[0])*self.loss(y, predicted_y) + a_2 * (1/f_o.shape[0])*self.loss( f_o, predicted_f_o) + a_3 *(1/s_o.shape[0])*self.loss(s_o,  predicted_s_o) 
    
    # logs
  #  self.log('val_loss', loss) # no optimization required for
  #  print( loss) 
    return {'val_loss' : loss}

  def validation_epoch_end(self, outputs):
    avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean() # calculates average loss for a epoch accounting for the batches
    tensorboard_logs = {'val_loss': avg_loss} # loggs on the graph for tensor boad 
    self.logger.experiment.add_scalar('val_loss',avg_loss, self.current_epoch) # logs loss against epochs. https://stackoverflow.com/questions/66290662/how-to-get-step-wise-validation-loss-curve-over-all-epochs-in-pytorch-lightning
    print('val_loss: ', avg_loss, "epoch_number",self.current_epoch) # self.current_epoch logs the information for the epoch.
    {'avg_val_loss': avg_loss,'progress_bar': tensorboard_logs}
  # https://forums.pytorchlightning.ai/t/how-to-get-step-wise-validation-loss-curve-over-all-epochs/752  - use self.logger.experiment instead of self.log 
    return {'avg_val_loss': avg_loss,'progress_bar': tensorboard_logs}

  def test_step(self, test_batch, batch_idx):
    x, y, f_o, s_o = test_batch
    predicted_y = self.forward(x).to(device)
    # loop over batch
    predicted_f_o = torch.zeros(size=(x.shape[0], 4)).to(device)
    for i in range(x.shape[0]): # validatio batch size
      predicted_f_o[i] = torch.autograd.functional.jacobian(self.forward, x[i], create_graph=True, strict=True)

    predicted_s_o = torch.zeros(size=(x.shape[0], 4, 4)).to(device)
    for i in range(x.shape[0]): # validation batch size
      predicted_s_o[i] = torch.autograd.functional.hessian(self.forward, x[i], create_graph=True, strict=True)

    loss = a_1 * (1/y.shape[0])*self.loss(y, predicted_y) + a_2 * (1/f_o.shape[0])*self.loss( f_o, predicted_f_o) + a_3 *(1/s_o.shape[0])*self.loss(s_o,  predicted_s_o) 

  #  self.log('test_loss', loss)
    
    return {'test_loss': loss}

  def test_epoch_end(self, outputs):
    avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean() # calculates average loss for a epoch accounting for the batches
    tensorboard_logs = {'test_loss': avg_loss} # loggs on the graph for tensor boad
  #  print('test_loss: ', avg_loss, "epoch_number",self.current_epoch) # self.current_epoch logs the information for the epoch.
          
    return {'avg_test_loss': avg_loss,'progress_bar': tensorboard_logs}
    
  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(), lr=5e-3) # increase learning rate


In [ ]:
trainer = pl.Trainer(gpus =1, max_epochs = 500) # Can now perform at 500 epochs neaded at most.
# https://www.machinecurve.com/index.php/question/how-to-set-number-of-epochs-in-pytorch-lightning/
model = SecondOrderNetwork().to(device) # Trained. 

trainer.fit(model, train_loader, val_loader)

# take lograithems 

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type     | Params
--------------------------------------
0 | layer_1  | Linear   | 1.3 K 
1 | layer_2  | Linear   | 32.9 K
2 | layer_3  | Linear   | 8.3 K 
3 | layer_4  | Linear   | 65    
4 | Softplus | Softplus | 0     
5 | loss     | MSELoss  | 0     
--------------------------------------
42.5 K    Trainable params
0         Non-trainable params
42.5 K    Total params
0.170     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

val_loss:  tensor(33.29041672, device='cuda:0') epoch_number 0


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:323: UserWarning: The number of training samples (24) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"


Training: -1it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

val_loss:  tensor(1.77453232, device='cuda:0') epoch_number 713
train_loss:  tensor(2.68188190, device='cuda:0') epoch_number 713


Validating: 0it [00:00, ?it/s]

val_loss:  tensor(3.25920177, device='cuda:0') epoch_number 714
train_loss:  tensor(5.89423943, device='cuda:0') epoch_number 714


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1047: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
 %reload_ext tensorboard 
 %tensorboard --logdir lightning_logs/
 #!kill 1983
#%reload_ext tensorboard
# https://stackoverflow.com/questions/51079373/automatically-save-tensorboard-like-plot-of-loss-to-image-file - how to save files, 






<IPython.core.display.Javascript object>